# ML Training with MSD Lung RadiObject: Segmentation

This notebook trains a 3D UNet for lung tumor segmentation using the Medical Segmentation Decathlon data.

## Overview

1. **Load** RadiObject from URI (S3 or local)
2. **Explore** data and segmentation masks
3. **Split** into train/validation sets
4. **Train** a MONAI UNet model
5. **Evaluate** with Dice score

## Task

Semantic segmentation: Predict lung tumor mask from CT volume patches.

**Prerequisites:** Run [05_ingest_msd.ipynb](./05_ingest_msd.ipynb) first to create the MSD Lung RadiObject with CT and seg collections.

In [ ]:
import shutil
import tempfile

import matplotlib.pyplot as plt
import numpy as np
import torch
from config import MSD_LUNG_URI, S3_REGION
from monai.losses import DiceCELoss
from monai.metrics import DiceMetric
from monai.networks.nets import UNet
from monai.transforms import Compose, RandFlipd

from radiobject import RadiObject, configure
from radiobject.ctx import S3Config
from radiobject.ml import (
    create_training_dataloader,
    create_validation_dataloader,
)

print(f"PyTorch version: {torch.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"RadiObject URI: {MSD_LUNG_URI}")

In [ ]:
# Determine compute device
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
elif torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")

print(f"Training device: {DEVICE}")

In [ ]:
# Configure S3 access if using S3 URI
if MSD_LUNG_URI.startswith("s3://"):
    configure(s3=S3Config(region=S3_REGION, max_parallel_ops=8))

TEMP_DIR = tempfile.mkdtemp(prefix="msd_ml_")
print(f"Temp directory: {TEMP_DIR}")

In [ ]:
# Load RadiObject
radi = RadiObject(MSD_LUNG_URI)

# Quick summary using describe()
print(radi.describe())

In [ ]:
# Verify both CT and seg collections exist
print(f"Collections: {radi.collection_names}")
print(f"CT shape: {radi.CT.shape}")
print(f"seg shape: {radi.seg.shape}")

if "seg" not in radi.collection_names:
    raise RuntimeError(
        "Segmentation collection not found. "
        "Please re-run 05_ingest_msd.ipynb with FORCE_REINGEST=True"
    )

In [ ]:
# Visualize CT and segmentation overlay for a few subjects
subject_ids = list(radi.obs_subject_ids)[:3]

fig, axes = plt.subplots(len(subject_ids), 3, figsize=(12, 4 * len(subject_ids)))

for row, subject_id in enumerate(subject_ids):
    ct_vol = radi.loc[subject_id].CT.iloc[0]
    seg_vol = radi.loc[subject_id].seg.iloc[0]

    # Find slice with tumor
    seg_data = seg_vol.to_numpy()
    tumor_slices = np.where(seg_data.sum(axis=(0, 1)) > 0)[0]
    mid_z = (
        tumor_slices[len(tumor_slices) // 2] if len(tumor_slices) > 0 else seg_data.shape[2] // 2
    )

    # CT
    axes[row, 0].imshow(ct_vol.axial(z=mid_z).T, cmap="gray", origin="lower")
    axes[row, 0].set_title(f"{subject_id} - CT")
    axes[row, 0].axis("off")

    # Segmentation
    axes[row, 1].imshow(seg_vol.axial(z=mid_z).T, cmap="hot", origin="lower")
    axes[row, 1].set_title(f"{subject_id} - Tumor Mask")
    axes[row, 1].axis("off")

    # Overlay
    axes[row, 2].imshow(ct_vol.axial(z=mid_z).T, cmap="gray", origin="lower")
    mask = seg_vol.axial(z=mid_z).T > 0
    axes[row, 2].imshow(np.ma.masked_where(~mask, mask), cmap="Reds", alpha=0.5, origin="lower")
    axes[row, 2].set_title(f"{subject_id} - Overlay")
    axes[row, 2].axis("off")

plt.tight_layout()
plt.show()

In [ ]:
# 80/20 split
all_ids = list(radi.obs_subject_ids)
np.random.seed(42)
np.random.shuffle(all_ids)

split_idx = int(0.8 * len(all_ids))
train_ids = all_ids[:split_idx]
val_ids = all_ids[split_idx:]

print(f"Training subjects: {len(train_ids)}")
print(f"Validation subjects: {len(val_ids)}")

In [ ]:
# Create train/val RadiObjects using loc[] and to_radi_object()
train_uri = f"{TEMP_DIR}/train_radi"
val_uri = f"{TEMP_DIR}/val_radi"

radi_train = radi.loc[train_ids].to_radi_object(train_uri)
radi_val = radi.loc[val_ids].to_radi_object(val_uri)

print(f"Train RadiObject: {radi_train}")
print(f"Val RadiObject: {radi_val}")

In [ ]:
# Training hyperparameters
BATCH_SIZE = 2
PATCH_SIZE = (64, 64, 64)

# Define transforms - only random flip (applies to both CT and seg consistently)
# Normalization is applied only to CT channel in training loop
train_transform = Compose(
    [
        RandFlipd(keys="image", prob=0.5, spatial_axis=[0, 1, 2]),
    ]
)

# Create dataloaders with multi-collection input
# [CT, seg] -> stacked as channels: batch["image"] shape (B, 2, D, H, W)
# Channel 0 = CT (input), Channel 1 = seg (label)
train_loader = create_training_dataloader(
    [radi_train.CT, radi_train.seg],  # Multi-collection: first=image, second=label
    batch_size=BATCH_SIZE,
    patch_size=PATCH_SIZE,
    num_workers=0,
    pin_memory=False,
    persistent_workers=False,
    transform=train_transform,
)

# Validation: no augmentation
val_loader = create_validation_dataloader(
    [radi_val.CT, radi_val.seg],
    batch_size=BATCH_SIZE,
    patch_size=PATCH_SIZE,
    num_workers=0,
    pin_memory=False,
)

print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")

In [ ]:
# Inspect a batch
batch = next(iter(train_loader))

print(f"Batch keys: {list(batch.keys())}")
print(f"Image shape: {batch['image'].shape}")  # (B, 2, D, H, W) - CT + seg stacked
print(f"Image dtype: {batch['image'].dtype}")
print(f"Memory per batch: {batch['image'].nbytes / 1024 / 1024:.1f} MB")

# Verify channel structure
ct_channel = batch["image"][:, 0:1, ...]  # CT
seg_channel = batch["image"][:, 1:2, ...]  # Segmentation mask
print(f"\nCT channel range: [{ct_channel.min():.2f}, {ct_channel.max():.2f}]")
print(f"Seg channel unique values: {torch.unique(seg_channel).tolist()}")

In [ ]:
# MONAI UNet for 3D segmentation
model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=2,  # background + tumor
    channels=(16, 32, 64, 128),
    strides=(2, 2, 2),
    num_res_units=2,
).to(DEVICE)

print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# Training configuration
NUM_EPOCHS = 10
LEARNING_RATE = 1e-3

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = DiceCELoss(to_onehot_y=True, softmax=True)
dice_metric = DiceMetric(include_background=False, reduction="mean")

# Training history
history = {
    "train_loss": [],
    "train_dice": [],
    "val_loss": [],
    "val_dice": [],
}


def normalize_ct(x: torch.Tensor) -> torch.Tensor:
    """Normalize CT intensities to zero mean, unit variance."""
    return (x - x.mean()) / (x.std() + 1e-8)

In [ ]:
print(f"Training on {DEVICE} for {NUM_EPOCHS} epochs...\n")

for epoch in range(NUM_EPOCHS):
    # Training phase
    model.train()
    train_loss = 0.0
    dice_metric.reset()

    for batch in train_loader:
        # Split stacked channels: CT (input) and seg (label)
        images = batch["image"][:, 0:1, ...].to(DEVICE)  # CT channel
        labels = batch["image"][:, 1:2, ...].long().to(DEVICE)  # seg channel

        # Normalize CT only (not seg mask)
        images = normalize_ct(images)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        # Compute Dice on predictions
        preds = torch.argmax(outputs, dim=1, keepdim=True)
        dice_metric(preds, labels)

    train_dice = dice_metric.aggregate().item()

    # Validation phase
    model.eval()
    val_loss = 0.0
    dice_metric.reset()

    with torch.no_grad():
        for batch in val_loader:
            images = batch["image"][:, 0:1, ...].to(DEVICE)
            labels = batch["image"][:, 1:2, ...].long().to(DEVICE)

            images = normalize_ct(images)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            preds = torch.argmax(outputs, dim=1, keepdim=True)
            dice_metric(preds, labels)

    val_dice = dice_metric.aggregate().item()

    # Record metrics
    history["train_loss"].append(train_loss / len(train_loader))
    history["train_dice"].append(train_dice)
    history["val_loss"].append(val_loss / len(val_loader))
    history["val_dice"].append(val_dice)

    print(
        f"Epoch {epoch + 1:2d}/{NUM_EPOCHS}: "
        f"Train Loss={history['train_loss'][-1]:.4f}, "
        f"Train Dice={history['train_dice'][-1]:.4f}, "
        f"Val Loss={history['val_loss'][-1]:.4f}, "
        f"Val Dice={history['val_dice'][-1]:.4f}"
    )

In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Loss
axes[0].plot(history["train_loss"], marker="o", label="Train")
axes[0].plot(history["val_loss"], marker="s", label="Validation")
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("Loss")
axes[0].set_title("Training & Validation Loss")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Dice Score
axes[1].plot(history["train_dice"], marker="o", label="Train")
axes[1].plot(history["val_dice"], marker="s", label="Validation")
axes[1].set_xlabel("Epoch")
axes[1].set_ylabel("Dice Score")
axes[1].set_title("Training & Validation Dice")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Final metrics
print("=" * 40)
print("Final Results")
print("=" * 40)
print(f"Best Train Dice: {max(history['train_dice']):.4f}")
print(f"Best Val Dice: {max(history['val_dice']):.4f}")
print(f"Final Train Loss: {history['train_loss'][-1]:.4f}")
print(f"Final Val Loss: {history['val_loss'][-1]:.4f}")

In [ ]:
# Visualize predictions on validation data
model.eval()
batch = next(iter(val_loader))
images = batch["image"][:, 0:1, ...].to(DEVICE)
labels = batch["image"][:, 1:2, ...].cpu().numpy()

with torch.no_grad():
    images_norm = normalize_ct(images)
    outputs = model(images_norm)
    preds = torch.argmax(outputs, dim=1, keepdim=True).cpu().numpy()

# Show first sample
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
mid_z = images.shape[4] // 2

# CT
axes[0].imshow(images[0, 0, :, :, mid_z].cpu().T, cmap="gray", origin="lower")
axes[0].set_title("Input CT")
axes[0].axis("off")

# Ground Truth
axes[1].imshow(labels[0, 0, :, :, mid_z].T, cmap="hot", origin="lower")
axes[1].set_title("Ground Truth")
axes[1].axis("off")

# Prediction
axes[2].imshow(preds[0, 0, :, :, mid_z].T, cmap="hot", origin="lower")
axes[2].set_title("Prediction")
axes[2].axis("off")

plt.suptitle(f"Validation Sample (z={mid_z})")
plt.tight_layout()
plt.show()

In [ ]:
# Cleanup temporary directory
shutil.rmtree(TEMP_DIR)
print(f"Cleaned up: {TEMP_DIR}")